# A/B 测试基础 - 从设计到分析

## 学习目标

完成本练习后，你将能够：
1. 理解随机化实验的原理，以及为什么它是因果推断的「黄金标准」
2. 掌握样本量计算和功效分析 (Power Analysis)
3. 学习分层随机化设计，提高实验效率
4. 了解 AA 测试的作用和常见问题诊断

## 真实场景：新版首页的转化率实验

想象你是某电商平台的增长负责人，设计师提出了新版首页设计，声称：

> "新版首页的转化率至少会提升 2%！"

作为数据驱动的决策者，你需要回答：

1. **能不能直接全量上线？** 万一新版更差怎么办？
2. **需要多少用户测试？** 100 人够吗？1000 人呢？
3. **怎么分组才公平？** 老用户 vs 新用户，安卓 vs iOS，是否需要分层？
4. **怎么判断效果？** p-value < 0.05 就一定显著吗？

这些问题的答案，都在 A/B 测试的科学设计中。

---

## Part 1: 随机化的原理

### 为什么随机化能识别因果效应？

回忆因果推断的根本问题：**同一个人，无法同时观测到「接受处理」和「不接受处理」的结果。**

$$\text{ITE}_i = Y_i(1) - Y_i(0)$$

但我们永远无法同时观测到 $Y_i(1)$ 和 $Y_i(0)$（反事实问题）。

**随机化的魔法：**

如果我们随机将用户分为两组：
- **实验组 (Treatment Group)**：看到新版首页
- **对照组 (Control Group)**：看到旧版首页

那么，两组用户在**所有维度上的分布都是相同的**（在期望意义上）：

- 年龄、性别、地域、消费能力
- 历史行为、浏览习惯、活跃度
- **甚至我们观测不到的特征**（这是关键！）

因此，实验组和对照组的**平均结果差异**，可以归因于**唯一的不同**：新版 vs 旧版首页。

$$
\text{ATE} = E[Y(1) - Y(0)] = E[Y | T=1] - E[Y | T=0]
$$

**核心理念：随机化让处理变量 $T$ 与所有潜在混淆因素独立。**

### 随机化 vs 非随机化

| 维度 | 随机化实验 (A/B Test) | 观测性研究 (Observational Study) |
|------|----------------------|----------------------------------|
| 混淆因素 | ✅ 自动平衡（包括不可观测的） | ❌ 需要手动控制（无法控制不可观测的） |
| 因果识别 | ✅ 无需假设 | ❌ 需要强假设 (CIA, Unconfoundedness) |
| 内部效度 | ✅ 高 | ⚠️ 可能有偏差 |
| 成本 | ⚠️ 高（需要实验环境） | ✅ 低（利用现有数据） |
| 伦理约束 | ⚠️ 可能有（医疗、教育） | ✅ 无 |

**生活中的比喻：**

- **随机化实验**：像掷硬币决定吃药 vs 不吃药，确保两组人完全可比
- **观测性研究**：像问「吃过药的人」vs「没吃过药的人」，可能吃药的人本身就更健康/更不健康

### 伪随机 vs 真随机

在实际工程中，我们通常使用**伪随机数生成器** (PRNG)：

```python
# 伪随机：基于 seed，可复现
import numpy as np
np.random.seed(42)
np.random.rand(5)  # 相同 seed → 相同结果
```

**优点：**
- 可复现（调试、审计）
- 基于 user_id hash，确保同一用户总是分到同一组

**真随机 vs 伪随机：**
- **真随机**：物理随机源（硬件噪声、放射性衰变），不可复现
- **伪随机**：确定性算法（Mersenne Twister, PCG），周期长、统计性质好

对 A/B 测试而言，**高质量的伪随机已经足够**。

---

## Part 2: 实验设计的关键要素

### 2.1 实验单元的选择

**实验单元 (Unit of Randomization)**: 随机化的最小单位。

| 场景 | 实验单元 | 原因 |
|------|----------|------|
| 网页 A/B 测试 | 用户 (user_id) | 避免同一用户看到不同版本 |
| 推荐算法实验 | 用户 (user_id) | 确保用户体验一致 |
| 定价实验 | 城市/区域 | 避免同城不同价引发投诉 |
| 教育干预实验 | 学校/班级 | 避免同班学生相互影响 (SUTVA 违背) |

**SUTVA 假设 (Stable Unit Treatment Value Assumption):**
- 个体 $i$ 的结果不受个体 $j$ 的处理影响
- 违背示例：社交网络（用户之间相互影响）、双边市场（司机-乘客）

**如果 SUTVA 违背怎么办？**
- 使用**聚类随机化** (Cluster Randomization)：以群组为单位随机化
- 详见后续章节《网络效应处理》

### 2.2 指标体系设计

#### 三层指标体系

```
┌─────────────────────────────────────┐
│  北极星指标 (North Star Metric)     │  ← 最关心的业务指标
│  例：GMV、DAU、留存率                │
└─────────────────────────────────────┘
           ↓
┌─────────────────────────────────────┐
│  护栏指标 (Guardrail Metrics)       │  ← 确保不伤害关键体验
│  例：页面加载时间、崩溃率、投诉率    │
└─────────────────────────────────────┘
           ↓
┌─────────────────────────────────────┐
│  诊断指标 (Diagnostic Metrics)      │  ← 帮助理解 Why
│  例：点击率、停留时长、跳出率        │
└─────────────────────────────────────┘
```

**决策逻辑：**
1. **北极星指标提升** + 护栏指标不恶化 → ✅ 上线
2. **北极星指标提升** + 护栏指标恶化 → ⚠️ 权衡决策
3. **北极星指标不显著** + 诊断指标提升 → 🤔 深入分析，可能是样本量不足
4. **北极星指标下降** → ❌ 不上线

### 2.3 分流比例的选择

**常见分流比例：**

| 比例 | 适用场景 | 优缺点 |
|------|----------|--------|
| 50:50 | 常规 A/B 测试 | ✅ 统计功效最高 |
| 90:10 | 风险较高的实验 | ⚠️ 需要更大样本量 |
| 95:5 | 灰度发布 | ⚠️ 功效低，主要用于风险控制 |

**数学原理：**

假设实验组比例为 $p$，对照组为 $1-p$，则方差为：

$$
\text{Var}(\hat{\text{ATE}}) \propto \frac{1}{p(1-p) \cdot n}
$$

- $p = 0.5$ 时，$p(1-p) = 0.25$ 最大，方差最小
- $p = 0.1$ 时，$p(1-p) = 0.09$，方差约为 50:50 的 **2.8 倍**

**推荐做法：**
- 低风险实验：50:50
- 高风险实验：先 95:5 灰度，确认安全后扩大到 50:50

---

## Part 3: 样本量计算与功效分析

### 3.1 核心概念

#### 假设检验框架

$$
\begin{align}
H_0 &: \mu_1 - \mu_0 = 0 \quad \text{(零假设：无效应)} \\
H_1 &: \mu_1 - \mu_0 \neq 0 \quad \text{(备择假设：有效应)}
\end{align}
$$

#### 两类错误

| 错误类型 | 定义 | 概率 | 控制手段 |
|----------|------|------|----------|
| Type I Error (假阳性) | 实际无效应，判断为有效应 | $\alpha$ (显著性水平) | 设置 $\alpha = 0.05$ |
| Type II Error (假阴性) | 实际有效应，判断为无效应 | $\beta$ | 提高样本量 |

**统计功效 (Power):**

$$
\text{Power} = 1 - \beta
$$

**直观理解：**
- **$\alpha = 0.05$**: 如果实际无效应，我们有 5% 的概率错误地认为有效应（狼来了）
- **Power = 0.8**: 如果实际有效应，我们有 80% 的概率能检测出来（灵敏度）

#### MDE (Minimum Detectable Effect)

**定义：** 在给定 $\alpha, \beta, n$ 下，能够检测到的最小效应量。

$$
\text{MDE} = \left( z_{1-\alpha/2} + z_{1-\beta} \right) \sqrt{\frac{2\sigma^2}{n}}
$$

**符号说明：**
- $z_{1-\alpha/2}$: 双侧检验的临界值 (例如 $\alpha=0.05$ 时为 1.96)
- $z_{1-\beta}$: 功效对应的 z 分数 (例如 Power=0.8 时为 0.84)
- $\sigma^2$: 结果变量的方差
- $n$: 每组样本量

**业务含义：**
- 如果你的产品转化率是 10%，MDE = 1%，那么小于 1% 的提升你检测不出来
- 想检测更小的效应 → 需要更大的样本量

### 3.2 样本量计算公式

对于**连续变量** (例如客单价、停留时长)：

$$
n = \frac{2(z_{1-\alpha/2} + z_{1-\beta})^2 \sigma^2}{\delta^2}
$$

对于**二元变量** (例如转化率、点击率)：

$$
n = \frac{2(z_{1-\alpha/2} + z_{1-\beta})^2 p(1-p)}{(p_1 - p_0)^2}
$$

其中：
- $\delta = \mu_1 - \mu_0$: 期望检测的效应量
- $p = (p_1 + p_0) / 2$: 平均转化率
- $p_1 - p_0$: 期望的转化率差异

### 环境准备

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats
from typing import Tuple, Dict

# 设置随机种子
np.random.seed(42)

# 颜色方案
COLORS = {
    'primary': '#2D9CDB',
    'success': '#27AE60',
    'danger': '#EB5757',
    'warning': '#F2994A',
    'neutral': '#828282',
}

print("✅ 环境准备完成")

### 3.3 实现样本量计算器

In [ ]:
# TODO 1: 实现转化率实验的样本量计算
# 提示：
# - 使用 stats.norm.ppf() 获取 z 分数
# - alpha=0.05 (双侧) 对应 z = stats.norm.ppf(1 - 0.05/2) = 1.96
# - power=0.8 对应 z = stats.norm.ppf(0.8) = 0.84
# - 公式: n = 2 * (z_alpha + z_beta)^2 * p_avg * (1 - p_avg) / delta^2

def calculate_sample_size_conversion(
    baseline_rate: float,
    mde: float,
    alpha: float = 0.05,
    power: float = 0.8
) -> int:
    """
    计算转化率实验所需的样本量
    
    参数:
        baseline_rate: 基线转化率 (例如 0.10 表示 10%)
        mde: 最小可检测效应 (例如 0.02 表示 2% 的绝对提升)
        alpha: 显著性水平 (Type I Error)
        power: 统计功效 (1 - Type II Error)
    
    返回:
        每组所需样本量
    """
    # 实现完成
    z_alpha = stats.norm.ppf(1 - alpha / 2)  # 双侧检验
    z_beta = stats.norm.ppf(power)
    p_avg = baseline_rate + mde / 2  # 平均转化率
    n_per_group = 2 * (z_alpha + z_beta)**2 * p_avg * (1 - p_avg) / mde**2
    
    return int(np.ceil(n_per_group))


# 测试函数
baseline = 0.10  # 基线转化率 10%
mde = 0.02       # 希望检测 2% 的提升
n = calculate_sample_size_conversion(baseline, mde)
print(f"基线转化率: {baseline*100:.1f}%")
print(f"MDE: {mde*100:.1f}%")
print(f"每组所需样本量: {n:,}")
print(f"总样本量: {2*n:,}")

In [ ]:
# 📝 参考答案（先自己尝试！）
# ================================
# z_alpha = stats.norm.ppf(1 - alpha / 2)  # 双侧检验
# z_beta = stats.norm.ppf(power)
# p_avg = baseline_rate + mde / 2  # 平均转化率
# n_per_group = 2 * (z_alpha + z_beta)**2 * p_avg * (1 - p_avg) / mde**2
# return int(np.ceil(n_per_group))

### 3.4 功效分析可视化

In [ ]:
# 生成不同 MDE 下的样本量需求
baseline = 0.10
mde_range = np.linspace(0.005, 0.05, 50)  # 0.5% ~ 5%
sample_sizes = [calculate_sample_size_conversion(baseline, mde) for mde in mde_range]

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=mde_range * 100,
    y=sample_sizes,
    mode='lines',
    name='样本量需求',
    line=dict(color=COLORS['primary'], width=3)
))

# 标注几个关键点
key_mdes = [0.01, 0.02, 0.03]
for key_mde in key_mdes:
    n = calculate_sample_size_conversion(baseline, key_mde)
    fig.add_annotation(
        x=key_mde * 100,
        y=n,
        text=f"MDE={key_mde*100:.1f}%<br>N={n:,}",
        showarrow=True,
        arrowhead=2,
        ax=40,
        ay=-40
    )

fig.update_layout(
    title='样本量需求 vs MDE（基线转化率 10%）',
    xaxis_title='最小可检测效应 MDE (%)',
    yaxis_title='每组所需样本量',
    template='plotly_white',
    height=500
)

fig.show()

print("💡 关键洞察：")
print("- MDE 越小，所需样本量呈指数增长")
print("- 想检测 1% 的提升，需要约 15,000 人/组")
print("- 想检测 2% 的提升，需要约 3,800 人/组")
print("- 实际业务中需要权衡：效应量的业务价值 vs 实验成本")

---

## Part 4: 分层随机化 (Stratified Randomization)

### 4.1 为什么需要分层？

**场景：** 你的用户包含「新用户」和「老用户」，他们的基线转化率差异很大：
- 新用户：5%
- 老用户：15%

**问题：** 如果完全随机分配，可能出现：
- 实验组：60% 新用户，40% 老用户
- 对照组：40% 新用户，60% 老用户

→ 即使新版本完全无效，实验组的转化率也会偏低（因为新用户占比高）

**分层随机化的思想：**
1. 先按「用户类型」分层
2. 在每层内独立随机分配
3. 确保实验组和对照组的「用户类型分布」完全一致

### 4.2 分层变量的选择

**好的分层变量：**
- 与结果变量**强相关**
- 在实验前已知（不能用实验后的行为分层）
- 类别数不能太多（否则每层样本量太小）

**常见分层变量：**

| 变量 | 适用场景 |
|------|----------|
| 新老用户 | 通用 |
| 平台 (iOS/Android) | App 实验 |
| 地域 | 地域相关产品 |
| 历史消费金额 (分桶) | 电商 |
| 活跃度 (DAU/MAU) | 社交产品 |

**分层 vs 事后分层 (Post-Stratification):**

| 维度 | 分层随机化 | 事后分层 |
|------|------------|----------|
| 时机 | 实验前 | 分析时 |
| 组间平衡 | ✅ 严格保证 | ⚠️ 依赖随机性 |
| 方差减小 | ✅ 更大 | ⚠️ 较小 |
| 实现难度 | ⚠️ 需要分流系统支持 | ✅ 分析时即可 |

### 4.3 实现分层随机化

In [ ]:
# TODO 2: 实现分层随机化分配函数
# 提示：
# - 对每个 stratum，独立进行随机分配
# - 使用 np.random.choice() 或 np.random.permutation()
# - 确保每层的实验组/对照组比例相同

def stratified_randomization(
    df: pd.DataFrame,
    strata_col: str,
    treatment_ratio: float = 0.5,
    seed: int = 42
) -> pd.DataFrame:
    """
    分层随机化分配
    
    参数:
        df: 用户数据
        strata_col: 分层变量列名
        treatment_ratio: 实验组比例
        seed: 随机种子
    
    返回:
        添加了 treatment 列的 DataFrame (1=实验组, 0=对照组)
    """
    np.random.seed(seed)
    df = df.copy()
    
    # 实现完成
    treatments = []
    for stratum in df[strata_col].unique():
        mask = df[strata_col] == stratum
        n_stratum = mask.sum()
        n_treatment = int(n_stratum * treatment_ratio)
        stratum_treatment = np.concatenate([
            np.ones(n_treatment),
            np.zeros(n_stratum - n_treatment)
        ])
        np.random.shuffle(stratum_treatment)
        treatments.append(stratum_treatment)
    df['treatment'] = np.concatenate(treatments)
    
    return df

In [ ]:
# 📝 参考答案（先自己尝试！）
# ================================
# treatments = []
# for stratum in df[strata_col].unique():
#     mask = df[strata_col] == stratum
#     n_stratum = mask.sum()
#     n_treatment = int(n_stratum * treatment_ratio)
#     stratum_treatment = np.concatenate([
#         np.ones(n_treatment),
#         np.zeros(n_stratum - n_treatment)
#     ])
#     np.random.shuffle(stratum_treatment)
#     treatments.append(stratum_treatment)
# df['treatment'] = np.concatenate(treatments)
# return df

In [ ]:
# 生成测试数据
def generate_user_data(n_users: int = 10000, seed: int = 42) -> pd.DataFrame:
    """
    生成用户数据
    """
    np.random.seed(seed)
    
    # 用户类型
    user_type = np.random.choice(['new', 'old'], size=n_users, p=[0.4, 0.6])
    
    # 基线转化率（新用户 5%，老用户 15%）
    baseline_conversion = np.where(user_type == 'new', 0.05, 0.15)
    
    df = pd.DataFrame({
        'user_id': range(n_users),
        'user_type': user_type,
        'baseline_conversion': baseline_conversion
    })
    
    return df

# 测试分层随机化
df_users = generate_user_data(10000)
df_assigned = stratified_randomization(df_users, 'user_type', treatment_ratio=0.5)

# 检查平衡性
balance_check = df_assigned.groupby(['user_type', 'treatment']).size().unstack(fill_value=0)
print("\n=== 分层随机化结果 ===")
print(balance_check)
print("\n各层的实验组比例:")
print(df_assigned.groupby('user_type')['treatment'].mean())

---

## Part 5: AA 测试 (AA Test)

### 5.1 AA 测试的作用

**定义：** 将用户随机分为两组，但**两组看到的内容完全相同**。

**目的：** 验证实验系统的正确性，诊断问题。

**AA 测试可以发现的问题：**

| 问题 | 表现 | 原因 |
|------|------|------|
| 分流不均匀 | 两组样本量差异大 | 分流算法问题 |
| 分流有偏 | 两组用户特征分布不同 | hash 函数有偏 |
| SRM (Sample Ratio Mismatch) | 实际分流比例 ≠ 预期 | 数据收集/过滤问题 |
| 假阳性率异常 | p < 0.05 的比例 >> 5% | 多重比较、数据质量问题 |

### 5.2 实现 AA 测试

In [ ]:
# TODO 3: 实现 AA 测试分析函数
# 提示：
# - 检查两组样本量是否接近 1:1
# - 对关键指标进行 t 检验或卡方检验
# - 计算 p-value < 0.05 的比例（应该接近 5%）

def aa_test_analysis(
    df: pd.DataFrame,
    treatment_col: str = 'treatment',
    metric_cols: list = None
) -> Dict:
    """
    AA 测试分析
    
    参数:
        df: 实验数据
        treatment_col: 分组列名
        metric_cols: 要检验的指标列
    
    返回:
        分析结果字典
    """
    results = {}
    
    # 1. 样本量检查
    sample_ratio = df[treatment_col].value_counts()
    results['sample_counts'] = sample_ratio.to_dict()
    results['sample_ratio'] = sample_ratio[1] / sample_ratio[0]
    
    # 2. SRM 检验 (Sample Ratio Mismatch)
    # 实现完成
    expected = np.array([sample_ratio.sum() / 2, sample_ratio.sum() / 2])
    observed = sample_ratio.values
    srm_stat, srm_pvalue = stats.chisquare(observed, expected)
    results['srm_pvalue'] = srm_pvalue
    
    # 3. 指标 t 检验
    if metric_cols is not None:
        metric_pvalues = {}
        for col in metric_cols:
            group_0 = df[df[treatment_col] == 0][col]
            group_1 = df[df[treatment_col] == 1][col]
            _, p_value = stats.ttest_ind(group_0, group_1)
            metric_pvalues[col] = p_value
        results['metric_pvalues'] = metric_pvalues
        results['significant_ratio'] = sum(p < 0.05 for p in metric_pvalues.values()) / len(metric_pvalues)
    
    return results


# 生成 AA 测试数据（两组完全相同）
def generate_aa_test_data(n_users: int = 5000, seed: int = 42) -> pd.DataFrame:
    np.random.seed(seed)
    df = pd.DataFrame({
        'user_id': range(n_users),
        'treatment': np.random.binomial(1, 0.5, n_users),
        'metric_1': np.random.normal(100, 15, n_users),
        'metric_2': np.random.normal(50, 10, n_users),
        'metric_3': np.random.binomial(1, 0.2, n_users),
    })
    return df

df_aa = generate_aa_test_data(5000)
aa_results = aa_test_analysis(df_aa, metric_cols=['metric_1', 'metric_2', 'metric_3'])

print("\n=== AA 测试结果 ===")
print(f"样本量: {aa_results['sample_counts']}")
print(f"样本比例: {aa_results['sample_ratio']:.3f} (预期: 1.0)")
print(f"SRM p-value: {aa_results.get('srm_pvalue', 'N/A'):.4f}")
print(f"\n各指标 p-value:")
for metric, pval in aa_results['metric_pvalues'].items():
    print(f"  {metric}: {pval:.4f}")
print(f"\n假阳性率: {aa_results['significant_ratio']*100:.1f}% (预期: ~5%)")

In [ ]:
# 📝 参考答案（先自己尝试！）
# ================================
# expected = np.array([sample_ratio.sum() / 2, sample_ratio.sum() / 2])
# observed = sample_ratio.values
# srm_stat, srm_pvalue = stats.chisquare(observed, expected)
# results['srm_pvalue'] = srm_pvalue

### 5.3 模拟多次 AA 测试

In [ ]:
# 模拟 1000 次 AA 测试
n_simulations = 1000
pvalues = []

for i in range(n_simulations):
    df_sim = generate_aa_test_data(5000, seed=i)
    group_0 = df_sim[df_sim['treatment'] == 0]['metric_1']
    group_1 = df_sim[df_sim['treatment'] == 1]['metric_1']
    _, p = stats.ttest_ind(group_0, group_1)
    pvalues.append(p)

# 可视化 p-value 分布
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=pvalues,
    nbinsx=20,
    name='p-value 分布',
    marker_color=COLORS['primary']
))

# 添加 p=0.05 参考线
fig.add_vline(
    x=0.05,
    line_dash="dash",
    line_color=COLORS['danger'],
    annotation_text="α = 0.05"
)

fig.update_layout(
    title=f'AA 测试 p-value 分布（{n_simulations} 次模拟）',
    xaxis_title='p-value',
    yaxis_title='频数',
    template='plotly_white'
)

fig.show()

false_positive_rate = sum(p < 0.05 for p in pvalues) / n_simulations
print(f"\n假阳性率: {false_positive_rate*100:.2f}% (理论值: 5.00%)")
print(f"p-value 在 [0, 0.05] 的比例: {false_positive_rate:.3f}")
print("\n💡 如果假阳性率远高于 5%，说明实验系统可能存在问题！")

---

## Part 6: 完整的 A/B 测试流程

### 6.1 端到端案例：电商首页改版实验

In [ ]:
# 业务场景设定
print("=== 业务背景 ===")
print("某电商平台准备改版首页，设计师声称新版能提升转化率")
print("\n关键参数:")
print("- 基线转化率: 10%")
print("- 目标 MDE: 2% (绝对提升)")
print("- 显著性水平: α = 0.05")
print("- 统计功效: Power = 0.8")
print("- 分流比例: 50:50")

# Step 1: 样本量计算
baseline_rate = 0.10
mde = 0.02
n_per_group = calculate_sample_size_conversion(baseline_rate, mde)
print(f"\n=== Step 1: 样本量计算 ===")
print(f"每组所需样本量: {n_per_group:,}")
print(f"总样本量: {2*n_per_group:,}")

# Step 2: 生成实验数据
def generate_ab_test_data(
    n_per_group: int,
    baseline_rate: float,
    true_effect: float,
    seed: int = 42
) -> pd.DataFrame:
    """
    生成 A/B 测试数据
    """
    np.random.seed(seed)
    n_total = 2 * n_per_group
    
    # 用户特征
    user_type = np.random.choice(['new', 'old'], size=n_total, p=[0.4, 0.6])
    
    # 分层随机化
    df = pd.DataFrame({
        'user_id': range(n_total),
        'user_type': user_type
    })
    df = stratified_randomization(df, 'user_type', treatment_ratio=0.5, seed=seed)
    
    # 生成结果（转化 0/1）
    # 新用户基线 5%，老用户基线 15%
    baseline = np.where(user_type == 'new', 0.05, 0.15)
    conversion_prob = baseline + df['treatment'] * true_effect
    df['converted'] = np.random.binomial(1, conversion_prob)
    
    return df

# 生成数据：假设真实效应是 +2.5%
true_effect = 0.025
df_experiment = generate_ab_test_data(n_per_group, baseline_rate, true_effect)

print(f"\n=== Step 2: 数据生成 ===")
print(f"实验数据: {len(df_experiment):,} 用户")
print(f"真实效应: +{true_effect*100:.1f}% (未知，仅用于验证)")

In [ ]:
# Step 3: 平衡性检查
print("\n=== Step 3: 平衡性检查 ===")
balance = df_experiment.groupby(['treatment', 'user_type']).size().unstack(fill_value=0)
print("\n用户类型分布:")
print(balance)
print("\n各组用户类型占比:")
print(balance.div(balance.sum(axis=1), axis=0).round(3))
print("\n✅ 分层随机化确保了两组用户类型分布完全一致")

In [ ]:
# TODO 4: 实现 A/B 测试分析函数
# 提示：
# - 计算两组的转化率和转化率差异
# - 使用双样本 t 检验或比例检验
# - 计算置信区间

def analyze_ab_test(df: pd.DataFrame) -> Dict:
    """
    A/B 测试分析
    
    参数:
        df: 实验数据，包含 treatment 和 converted 列
    
    返回:
        分析结果字典
    """
    results = {}
    
    # 实现完成
    # 1. 计算两组转化率
    control = df[df['treatment'] == 0]['converted']
    treatment = df[df['treatment'] == 1]['converted']
    
    control_rate = control.mean()
    treatment_rate = treatment.mean()
    
    # 2. 计算差异
    lift = treatment_rate - control_rate
    
    # 3. 统计检验（使用 stats.ttest_ind）
    _, p_value = stats.ttest_ind(treatment, control)
    
    # 4. 置信区间（使用正态近似）
    n_control = len(control)
    n_treatment = len(treatment)
    se = np.sqrt(treatment_rate * (1 - treatment_rate) / n_treatment +
                 control_rate * (1 - control_rate) / n_control)
    ci_lower = lift - 1.96 * se
    ci_upper = lift + 1.96 * se
    
    results['control_rate'] = control_rate
    results['treatment_rate'] = treatment_rate
    results['lift'] = lift
    results['p_value'] = p_value
    results['ci_lower'] = ci_lower
    results['ci_upper'] = ci_upper
    
    return results


# Step 4: 假设检验
results = analyze_ab_test(df_experiment)
print("\n=== Step 4: 假设检验 ===")
print(f"对照组转化率: {results.get('control_rate', 0)*100:.2f}%")
print(f"实验组转化率: {results.get('treatment_rate', 0)*100:.2f}%")
print(f"转化率提升: {results.get('lift', 0)*100:.2f}%")
print(f"95% 置信区间: [{results.get('ci_lower', 0)*100:.2f}%, {results.get('ci_upper', 0)*100:.2f}%]")
print(f"p-value: {results.get('p_value', 1):.4f}")
print(f"\n结论: {'✅ 拒绝原假设，效应显著' if results.get('p_value', 1) < 0.05 else '❌ 无法拒绝原假设'}")

In [ ]:
# 📝 参考答案（先自己尝试！）
# ================================
# control = df[df['treatment'] == 0]['converted']
# treatment = df[df['treatment'] == 1]['converted']
# 
# control_rate = control.mean()
# treatment_rate = treatment.mean()
# lift = treatment_rate - control_rate
# 
# _, p_value = stats.ttest_ind(control, treatment)
# 
# results['control_rate'] = control_rate
# results['treatment_rate'] = treatment_rate
# results['lift'] = lift
# results['p_value'] = p_value
# 
# return results

In [ ]:
# Step 5: 分层分析
print("\n=== Step 5: 分层分析 ===")
for user_type in ['new', 'old']:
    df_sub = df_experiment[df_experiment['user_type'] == user_type]
    control_rate = df_sub[df_sub['treatment'] == 0]['converted'].mean()
    treatment_rate = df_sub[df_sub['treatment'] == 1]['converted'].mean()
    lift = treatment_rate - control_rate
    print(f"\n{user_type.capitalize()} 用户:")
    print(f"  对照组转化率: {control_rate*100:.2f}%")
    print(f"  实验组转化率: {treatment_rate*100:.2f}%")
    print(f"  提升: {lift*100:+.2f}%")

### 6.2 可视化实验结果

In [ ]:
# 转化率对比图
summary = df_experiment.groupby('treatment')['converted'].agg(['mean', 'sem']).reset_index()
summary['treatment'] = summary['treatment'].map({0: '对照组', 1: '实验组'})

fig = go.Figure()

fig.add_trace(go.Bar(
    x=summary['treatment'],
    y=summary['mean'] * 100,
    error_y=dict(type='data', array=summary['sem'] * 100 * 1.96),  # 95% CI
    marker_color=[COLORS['neutral'], COLORS['success']],
    text=[f"{v*100:.2f}%" for v in summary['mean']],
    textposition='outside'
))

fig.update_layout(
    title='转化率对比（含 95% 置信区间）',
    yaxis_title='转化率 (%)',
    template='plotly_white',
    showlegend=False,
    height=400
)

fig.show()

In [ ]:
# 分层对比图
strata_summary = df_experiment.groupby(['user_type', 'treatment'])['converted'].mean().reset_index()
strata_summary['treatment'] = strata_summary['treatment'].map({0: '对照组', 1: '实验组'})

fig = px.bar(
    strata_summary,
    x='user_type',
    y='converted',
    color='treatment',
    barmode='group',
    title='分层转化率对比',
    labels={'user_type': '用户类型', 'converted': '转化率', 'treatment': '组别'},
    color_discrete_map={'对照组': COLORS['neutral'], '实验组': COLORS['success']}
)

fig.update_layout(
    template='plotly_white',
    yaxis_tickformat='.1%',
    height=400
)

fig.show()

print("\n💡 分层分析的价值:")
print("- 可以发现异质性效应（不同人群的效应可能不同）")
print("- 帮助制定精细化策略（例如只对新用户上线新版本）")

---

## 练习：设计你自己的 A/B 测试

### 场景：优化推荐算法

假设你是某视频平台的推荐算法负责人，想测试新推荐算法的效果。

**已知信息：**
- 基线点击率：20%
- 希望检测的最小提升：1.5% (绝对值)
- 用户分为「重度用户」和「轻度用户」
- 预算：可以对 20,000 用户进行实验

**任务：**
1. 计算所需样本量，判断预算是否足够
2. 设计分层随机化方案
3. 生成模拟数据并进行分析
4. 可视化结果

In [ ]:
# TODO 5: 完成练习
# 提示：
# - 使用前面实现的 calculate_sample_size_conversion()
# - 使用 stratified_randomization() 进行分层
# - 生成数据时考虑重度/轻度用户的基线差异
# - 使用 analyze_ab_test() 分析结果

# === 你的代码 ===




# === 代码结束 ===

---

## 思考题

### 基础理解
1. **为什么随机化能解决「混淆偏差」？** 它和观测性研究的 PSM/IPW 有什么本质区别？
2. **如果 p-value = 0.06，应该怎么决策？** 是「接近显著」还是「不显著」？
3. **为什么需要预先确定样本量？** 能不能「边跑边看，够了就停」？

### 深入分析
4. **多重比较问题：** 如果你同时测试 10 个指标，每个指标 $\alpha=0.05$，那么至少有一个假阳性的概率是多少？如何修正？
5. **分层随机化 vs 事后分层：** 如果忘记在实验前分层，事后分析时再分层，效果会差多少？
6. **不平衡分流（90:10）的场景：** 什么时候应该使用不平衡分流？它对统计功效有什么影响？

### 实战应用
7. **网络效应场景：** 假设你在做一个社交产品的实验，用户 A 的行为会影响其好友用户 B。此时简单随机化会有什么问题？如何改进？
8. **长期效应：** 实验只跑了 2 周，但你关心的是 3 个月后的留存率。如何设计实验？
9. **假设你发现实验组转化率提升了 2%，p < 0.01，非常显著。但同时发现页面加载时间增加了 500ms。你会上线吗？为什么？**

### 思维拓展
10. **A/B 测试 vs 因果推断：** A/B 测试估计的是什么效应（ATE/ATT/CATE）？它的外部效度如何？

---

## 本章小结

| 主题 | 核心要点 | 公式/方法 |
|------|----------|----------|
| **随机化原理** | 让处理变量 $T$ 与所有混淆因素独立，包括不可观测的 | $E[Y(1) - Y(0)] = E[Y \| T=1] - E[Y \| T=0]$ |
| **样本量计算** | 平衡 Type I Error ($\alpha$) 和 Type II Error ($\beta$) | $n = \frac{2(z_{\alpha} + z_{\beta})^2 \sigma^2}{\delta^2}$ |
| **MDE** | 给定样本量下能检测的最小效应 | MDE 越小 → 样本量越大（指数关系） |
| **分层随机化** | 在每层内独立随机化，确保组间协变量平衡 | 提高功效，减小方差 |
| **AA 测试** | 验证实验系统正确性 | 假阳性率应接近 $\alpha$ (通常 5%) |
| **指标体系** | 北极星指标 + 护栏指标 + 诊断指标 | 多维度评估，避免单一指标优化 |

### 关键决策框架

```
实验设计
├── 1. 定义目标指标（北极星 + 护栏）
├── 2. 计算样本量（基于 MDE）
├── 3. 选择实验单元（避免 SUTVA 违背）
├── 4. 分层随机化（如果有强协变量）
├── 5. AA 测试（验证系统）
├── 6. AB 测试（收集数据）
├── 7. 分析与决策
│   ├── 整体效应 + 分层效应
│   ├── 检查护栏指标
│   └── 业务价值评估（ROI）
└── 8. 上线决策
```

### 常见陷阱

| 陷阱 | 后果 | 避免方法 |
|------|------|----------|
| 提前停止（Peeking） | 假阳性率膨胀 | 预先确定样本量和停止规则 |
| 多重比较 | 假阳性率膨胀 | Bonferroni 校正或 FDR 控制 |
| SUTVA 违背 | ATE 估计有偏 | 聚类随机化 |
| SRM (分流比例异常) | 结果不可信 | AA 测试诊断 |
| 只看 p-value | 忽略效应大小和业务价值 | 同时看置信区间和业务 ROI |

### 下一步

- **Part 1.2: CUPED 方差缩减** - 如何用历史数据提高实验功效
- **Part 1.4: 网络效应处理** - SUTVA 违背时的实验设计
- **Part 1.6: 长期效应估计** - 用短期实验预测长期影响

---

## 延伸阅读

### 经典论文
1. **Kohavi et al. (2013)** - "Online Controlled Experiments at Large Scale" (Microsoft)
2. **Deng et al. (2017)** - "Improving the Sensitivity of Online Controlled Experiments" (LinkedIn CUPED)
3. **Xu et al. (2015)** - "Infrastructure for A/B Testing" (Facebook)

### 工业实践
- [Booking.com - How Booking.com increases the power of online experiments](https://booking.ai/)
- [Netflix - A/B Testing at Netflix](https://netflixtechblog.com/)
- [Airbnb - Experiments at Airbnb](https://medium.com/airbnb-engineering)

### 工具推荐
- [statsmodels.stats.power](https://www.statsmodels.org/stable/stats.html#power-and-sample-size-calculations) - Python 功效分析
- [Evan's Awesome A/B Tools](https://www.evanmiller.org/ab-testing/) - 在线计算器
- [GrowthBook](https://www.growthbook.io/) - 开源 A/B 测试平台